# Standard Errors

This notebook displays how the usual Ordinary Least Squares (OLS) estimator for the standard error of slope $\hat{\beta}$, $SE(\hat{\beta})$ fails to be an accurate and consistent estimator, when the error $\epsilon$ from the model $Y = \beta X + \epsilon$ is not homoscedastic (consistent variance for $\epsilon$ across different values of X). 
For heteroscedastic (inconsistent variance) or clustered data (each cluster has a distinct variance for the errors), 

- heteroscedasticity-consistent (https://en.wikipedia.org/wiki/Heteroscedasticity-consistent_standard_errors) estimators or 
- cluster-robust (http://cameron.econ.ucdavis.edu/research/Cameron_Miller_JHR_2015_February.pdf) estimators should be used instead.

To create the visualization, click on the code block below, and run that block by clicking on the 'run cell' button marked with red on the following image.

![title](Menu.PNG)

Running the code cell for the first time may take a while. Once the run is complete, you'll see some slidebars/dropdown menus along with the 'Run Interact' button. Adjust the parameters, and then press the button to see the visualization.

In [1]:
import ipywidgets as widgets
from visfunctions import SE
from ipywidgets import interact_manual
        
interact_manual(SE, option=widgets.Dropdown(options=['Homoscedastic', 'Heteroscedastic', 'Clustered'], description="data type:"))

C:\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Widget Javascript not detected.  It may not be installed or enabled properly.


<function visfunctions.SE>

- For a homoscedastic data, the OLS estimator for $SE(\hat{\beta})$ is an accurate estimate.
- For a heteroscedastic data, the OLS estimator does not provide a consistent estimate of the true variance. Instead, the heteroscedasticity-consistent estimate should be used. (Observe the difference between an OLS estimate and the heteroscedasticity-consistent estimate)
- For a clustered data, using non-cluster-robust estimate tends to underestimate the true variance of $SE(\hat{\beta})$. Instead, a cluster-robust estimate provides a better estimate. (Observe that the first two estimates wrongly estimate the standard error to be smaller, compared to the cluster-robust estimate)